# First training using the simple negation data

In [17]:
# all imports
import numpy as np
import pandas as pd
import torch
from torchtext import data
import torch.optim as optim

import os
import inspect
import sys
import itertools

almost_current = os.path.abspath(inspect.getfile(inspect.currentframe()))
currentdir = os.path.dirname(almost_current)
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir)

from plots.functions  import plot_confusion_matrix, plot_histogram_from_labels
from text_processing.functions import simple_pre_process_text_df, simple_pre_process_text

% matplotlib inline

In [22]:
train_data_path = os.path.join(parentdir,
                               "text_generator",
                                "b1_train.csv")

test_data_path = os.path.join(parentdir,
                              "text_generator",
                              "b1_test.csv")

In [24]:
train_data = pd.read_csv(train_data_path)
test_data = pd.read_csv(test_data_path)

In [25]:
train_data.head()

,sentence1,sentence2,label
0,Winston is lively,Brooklyn is not nice,0
1,Reynold is witty,Quanda is not thankful,0
2,Hazel is brave,Charity is not witty,0
3,Sutcliff is silly,Lyle is not ambitious,0
4,Banner is ambitious,Banner is not ambitious,1


In [21]:
test_data.head()

,sentence1,sentence2,label
0,Winston is lively,Brooklyn is not nice,0
1,Reynold is witty,Quanda is not thankful,0
2,Hazel is brave,Charity is not witty,0
3,Sutcliff is silly,Lyle is not ambitious,0
4,Banner is ambitious,Banner is not ambitious,1
